## Interpolasi Lagrange

Interpolasi Lagrange adalah metode untuk menentukan polinomial yang tepat melewati seluruh titik data secara langsung. Polinomial interpolasi Lagrange dinyatakan sebagai:
$$
P(x) = \sum_{i=0}^{n} y_i L_i(x)
$$
dengan \( L_i(x) \) adalah **polinomial basis Lagrange** yang didefinisikan sebagai:
$$
L_i(x) = \frac{(x - x_0)(x - x_1)\dots(x - x_{i-1})(x - x_{i+1})\dots(x - x_n)}{(x_i - x_0)(x_i - x_1)\dots(x_i - x_{i-1})(x_i - x_{i+1})\dots(x_i - x_n)}
$$

Interpolasi Lagrange tidak secara eksplisit menyelesaikan bentuk matriks, namun secara implisit memenuhi persamaan matriks berikut:
$$
\begin{bmatrix}
1 & x_0 & x_0^2 & \dots & x_0^n \\
1 & x_1 & x_1^2 & \dots & x_1^n \\
\vdots & \vdots & \vdots & \dots & \vdots \\
1 & x_n & x_n^2 & \dots & x_n^n
\end{bmatrix}
\begin{bmatrix}
a_0 \\ a_1 \\ \vdots \\ a_n
\end{bmatrix}
=
\begin{bmatrix}
y_0 \\ y_1 \\ \vdots \\ y_n
\end{bmatrix}
$$

Koefisien $a_0, a_1, \dots, a_n$ yang didapatkan dari persamaan matriks ini akan sesuai dengan polinomial yang dihasilkan metode Lagrange. Contoh implementasi diberikan berikut ini.

In [5]:
def lagrange_interp(x_pts, y_pts, x):
    n = len(x_pts)
    result = 0
    for i in range(n):
        term = y_pts[i]
        for j in range(n):
            if i != j:
                term *= (x - x_pts[j]) / (x_pts[i] - x_pts[j])
        result += term
    return result

# Contoh kasus (data gerak mobil di submodul sebleumnya):
t_pts = [0, 1, 3, 4]
x_pts = [0, 10, 35, 50]

# Interpolasi posisi pada t = 2
x_lagrange = lagrange_interp(t_pts, x_pts, 2)
print("Hasil interpolasi Lagrange pada t = 2 adalah", x_lagrange)

Hasil interpolasi Lagrange pada t = 2 adalah 21.666666666666668


## Interpolasi Newton

Interpolasi Newton menggunakan aproksimasi bertahap pada setiap titik data dengan konsep **selisih terbagi** atau **beda terbagi** (*divided differences*) untuk membangun polinomial interpolasi secara iteratif:
$$
P(x) = f[x_0] + f[x_0, x_1](x - x_0) + f[x_0, x_1, x_2](x - x_0)(x - x_1) + \dots
$$
Bentuk umumnya adalah:
$$
P(x) = f[x_0] + \sum_{i=1}^{n} f[x_0,x_1,\dots,x_i]\prod_{j=0}^{i-1}(x - x_j)
$$

Beda terbagi didefinisikan secara rekursif sebagai berikut:
- Beda terbagi orde pertama:
  $$
  f[x_i, x_{i+1}] = \frac{f(x_{i+1}) - f(x_i)}{x_{i+1} - x_i}
  $$

- Beda terbagi orde lebih tinggi:
  $$
  f[x_0,\dots,x_n] = \frac{f[x_1,\dots,x_n] - f[x_0,\dots,x_{n-1}]}{x_n - x_0}
  $$

Interpolasi Newton secara implisit menyelesaikan sistem persamaan matriks yang serupa dengan interpolasi Lagrange, yaitu:
$$
\begin{bmatrix}
1 & x_0 & x_0^2 & \dots & x_0^n \\
1 & x_1 & x_1^2 & \dots & x_1^n \\
\vdots & \vdots & \vdots & \dots & \vdots \\
1 & x_n & x_n^2 & \dots & x_n^n
\end{bmatrix}
\begin{bmatrix}
a_0 \\ a_1 \\ \vdots \\ a_n
\end{bmatrix}
=
\begin{bmatrix}
y_0 \\ y_1 \\ \vdots \\ y_n
\end{bmatrix}
$$

Metode beda terbagi Newton adalah cara efisien dan terstruktur untuk menghitung koefisien-koefisien tersebut secara bertahap. Contoh implementasi Python diberikan berikut ini.

In [6]:
def newton_interp(x_pts, y_pts, x):
    n = len(x_pts)
    coef = [0]*n

    # Menghitung beda terbagi
    for i in range(n):
        coef[i] = y_pts[i]
    for j in range(1, n):
        for i in range(n-1, j-1, -1):
            coef[i] = (coef[i] - coef[i-1]) / (x_pts[i] - x_pts[i-j])

    # Evaluasi polinomial Newton pada x
    result = coef[n-1]
    for i in range(n-2, -1, -1):
        result = result*(x - x_pts[i]) + coef[i]

    return result

# Contoh kasus (data gerak mobil):
t_pts = [0, 1, 3, 4]
x_pts = [0, 10, 35, 50]

# Interpolasi posisi pada t = 2
x_newton = newton_interp(t_pts, x_pts, 2)
print("Hasil interpolasi Newton pada t = 2 adalah", x_newton)

Hasil interpolasi Newton pada t = 2 adalah 21.666666666666668


## Komentar untuk hasil interpolasi Lagrange vs. Newton:

Kedua metode dalam kasus ini memberikan hasil yang sama karena polinomialnya sama persis. Namun, tidak serta merta semua kasus bisa begitu. Untuk perbandingan secara umum antara keduanya diberikan pada tabel berikut.

| Metode | Kelebihan | Kekurangan |
|--------|-----------|------------|
| **Lagrange** | - Langsung menghitung hasil akhir <br> - Tidak perlu menyelesaikan sistem persamaan linear secara eksplisit | - Berat secara komputasi untuk data besar <br> - Sulit diperbarui jika ada tambahan titik data baru |
| **Newton** | - Mudah diperbarui saat ada tambahan data baru <br> - Efisien karena koefisien dihitung secara bertahap (beda terbagi) | - Penurunan awal lebih kompleks dan kurang intuitif |

Kedua metode sama-sama efektif, tetapi Newton lebih disukai jika data akan ditambahkan secara bertahap, sementara Lagrange lebih sederhana untuk dataset kecil yang sudah tetap.